In [ ]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
SEED = 0
DROP_APPROACHES = [
    'BinSeg Clinear',
]
EXPECTED_NUMBER_APPROACHES = (3 * 10) + 1 - len(DROP_APPROACHES)

In [4]:
def get_df_for_dataset(database_name: str, dataset_name: str) -> pd.DataFrame:
    def get_df_for_dataset_inner(database_name: str, dataset_name: str) -> pd.DataFrame:
        directory_path = f"outputs/report/seed={SEED}/{database_name}/{dataset_name}"

        print(f"Reading from {directory_path}")

        metrics_dfs = []
        
        for root, dirs, files in os.walk(directory_path):
            for filename in files:
                if filename.endswith('.json'):
                    file_path = os.path.join(root, filename)
                    
                    try:
                        with open(file_path, 'r') as file:
                            data = json.load(file)
                            metrics_dfs.append(
                                pd.DataFrame({
                                    'cut_point_model': [data['cut_point_model']],
                                    'cut_point_method': [data['cut_point_method']],
                                    'cut_point_approach': [data['cut_point_approach']],
                                    'cut_point_perc': [data['cut_point_perc']],
                                    'cut_duration': [data['cut_duration']],
                                    'cut_duration_minutes': [data['cut_duration'] / 60],
                                    'tuner_duration': [data['tuner_duration']],
                                    'tuner_duration_minutes': [data['tuner_duration'] / 60],
                                    'retrain_duration': [data['retrain_duration']],
                                    'retrain_duration_minutes': [data['retrain_duration'] / 60],
                                    'total_duration': [data['total_duration']],
                                    'total_duration_minutes': [data['total_duration'] / 60],
                                    'Avg_MAPE': [data['error_results']['Avg_MAPE']],
                                    'Avg_MAE': [data['error_results']['Avg_MAE']], 
                                    'Avg_MSE': [data['error_results']['Avg_MSE']], 
                                    'Avg_RMSE': [data['error_results']['Avg_RMSE']], 
                                    'Avg_R2':  [data['error_results']['Avg_R2']],
                                    'Avg_WAPE':  [data['error_results']['Avg_WAPE']],
                                }))
                    except KeyError as e:
                        print(f"Error in {file_path}: {e}")
                        continue
                    
        metrics_df = pd.concat(metrics_dfs, axis=0)
    
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Rbf', 'RBF')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace(' Ar', ' AR')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bin_Seg', 'BinSeg')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bottom_Up', 'BottomUp')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_0.0', 'Full')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_', 'Fixed Cut ')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.1', '10%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.2', '20%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.3', '30%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.4', '40%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.5', '50%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.6', '60%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.7', '70%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.8', '80%')

        metrics_df = metrics_df[~metrics_df['cut_point_approach'].str.endswith(tuple(DROP_APPROACHES))]

        return metrics_df

    metrics_df = get_df_for_dataset_inner(
        database_name, dataset_name
    )

    metrics_cols = metrics_df.columns.tolist()
    metrics_df['database'] = [database_name] * len(metrics_df)
    metrics_df['dataset'] = [dataset_name] * len(metrics_df)
    metrics_df = metrics_df[['database', 'dataset'] + metrics_cols]

    return metrics_df.reset_index(drop=True)

In [5]:
datasets = []

# [UCI] APPLIANCES_ENERGY

In [6]:
uci_appliances_energy = get_df_for_dataset('UCI', 'APPLIANCES_ENERGY')
uci_appliances_energy_name = 'UCI APPLIANCES_ENERGY'
assert not uci_appliances_energy['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_appliances_energy) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_appliances_energy)}"

datasets.append(uci_appliances_energy)

uci_appliances_energy.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=0/UCI/APPLIANCES_ENERGY


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
19,UCI,APPLIANCES_ENERGY,Window,Clinear,Window Clinear,60.140613,0.065912,1.098529e-03,710.849442,11.847491,6.327037,0.105451,717.242391,11.954040,1.126937e+12,1.959334,9.996158,2.663869,0.827511,0.013224
2,UCI,APPLIANCES_ENERGY,Bottom_Up,L2,BottomUp L2,51.748163,0.058456,9.742618e-04,892.933278,14.882221,6.158123,0.102635,899.149857,14.985831,1.677983e+12,2.235957,11.206695,2.930961,0.772561,0.015091
14,UCI,APPLIANCES_ENERGY,Window,AR,Window AR,60.583988,0.109772,1.829533e-03,534.705817,8.911764,4.312757,0.071879,539.128346,8.985472,2.144749e+12,2.192368,11.667831,2.932543,0.788316,0.014797
27,UCI,APPLIANCES_ENERGY,Bin_Seg,L1,BinSeg L1,68.184697,2.278780,3.797967e-02,531.585353,8.859756,4.801790,0.080030,538.665923,8.977765,1.720601e+12,2.202004,12.178966,2.939858,0.790934,0.014862
8,UCI,APPLIANCES_ENERGY,Bottom_Up,Rank,BottomUp Rank,53.870028,0.126074,2.101231e-03,959.735475,15.995591,6.532726,0.108879,966.394275,16.106571,1.775468e+12,2.251709,11.522273,2.941116,0.783150,0.015197
5,UCI,APPLIANCES_ENERGY,Bottom_Up,Normal,BottomUp Normal,63.624272,0.094689,1.578152e-03,656.528812,10.942147,5.046623,0.084110,661.670124,11.027835,1.463742e+12,2.283389,12.285784,2.999015,0.788788,0.015411
29,UCI,APPLIANCES_ENERGY,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000008,1.311302e-07,1166.266474,19.437775,17.803534,0.296726,1184.070016,19.734500,2.232337e+12,2.685288,15.130140,3.491052,0.692681,0.018124
9,UCI,APPLIANCES_ENERGY,Bottom_Up,Clinear,BottomUp Clinear,1.330124,0.050056,8.342663e-04,1439.760000,23.996000,18.004424,0.300074,1457.814480,24.296908,2.764518e+12,2.759958,17.661080,3.698043,0.680086,0.018628
6,UCI,APPLIANCES_ENERGY,Bottom_Up,Mahalanobis,BottomUp Mahalanobis,23.498860,0.813616,1.356027e-02,1363.106299,22.718438,15.758885,0.262648,1379.678800,22.994647,1.962825e+12,2.879198,16.845162,3.734481,0.627951,0.019433
10,UCI,APPLIANCES_ENERGY,Window,Cosine,Window Cosine,60.298961,1.005344,1.675573e-02,762.478649,12.707977,17.957756,0.299296,781.441749,13.024029,1.826067e+12,2.874571,18.200538,3.746452,0.653446,0.019401


# [UCI] METRO_TRAFFIC

In [7]:
uci_metro_traffic = get_df_for_dataset('UCI', 'METRO_TRAFFIC')
uci_metro_traffic_name = 'UCI METRO_TRAFFIC'
assert not uci_metro_traffic['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_metro_traffic) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_metro_traffic)}"

datasets.append(uci_metro_traffic)

uci_metro_traffic.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=0/UCI/METRO_TRAFFIC
Error in outputs/report/seed=0/UCI/METRO_TRAFFIC/Bin_Seg/Cosine/2025-03-01_09-47-59/report.json: 'cut_point_perc'


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
27,UCI,METRO_TRAFFIC,Bin_Seg,L1,BinSeg L1,14.197547,8.355084,1.392514e-01,1912.284044,31.871401,40.516398,0.675273,1961.155526,32.685925,2.325033e+15,11.184320,332.255920,13.403232,-3.974085,0.102361
22,UCI,METRO_TRAFFIC,Bin_Seg,L2,BinSeg L2,14.197547,1.234178,2.056963e-02,1994.783680,33.246395,44.284218,0.738070,2040.302075,34.005035,2.325033e+15,11.184320,332.255920,13.403232,-3.974085,0.102361
23,UCI,METRO_TRAFFIC,Bin_Seg,RBF,BinSeg RBF,14.197547,4760.237926,7.933730e+01,2043.114689,34.051911,45.370736,0.756179,6848.723351,114.145389,2.325033e+15,11.184320,332.255920,13.403232,-3.974085,0.102361
20,UCI,METRO_TRAFFIC,Bin_Seg,Cosine,BinSeg Cosine,0.038897,29485.709741,4.914285e+02,2358.372987,39.306216,279.146289,4.652438,32123.229017,535.387150,1.974370e+15,11.151212,339.564114,13.518260,-2.306810,0.102058
29,UCI,METRO_TRAFFIC,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000008,1.311302e-07,2510.864900,41.847748,322.482454,5.374708,2833.347362,47.222456,2.205620e+15,11.000805,339.609517,13.568049,-3.071305,0.100681
10,UCI,METRO_TRAFFIC,Window,Cosine,Window Cosine,6.923735,26.671981,4.445330e-01,2543.120545,42.385342,62.555310,1.042588,2632.347836,43.872464,2.849866e+15,11.208098,333.732924,13.581176,-5.879762,0.102579
19,UCI,METRO_TRAFFIC,Window,Clinear,Window Clinear,31.117911,0.156389,2.606483e-03,1571.377104,26.189618,37.836628,0.630610,1609.370121,26.822835,2.700212e+15,11.417654,335.628635,13.602602,-5.530939,0.104497
21,UCI,METRO_TRAFFIC,Bin_Seg,Linear,BinSeg Linear,70.300547,4.428323,7.380538e-02,961.554894,16.025915,20.568828,0.342814,986.552045,16.442534,1.400426e+14,11.425263,360.694065,13.638690,0.310013,0.104566
25,UCI,METRO_TRAFFIC,Bin_Seg,Normal,BinSeg Normal,70.300547,2.877344,4.795574e-02,810.613239,13.510221,14.556372,0.242606,828.046955,13.800783,1.400426e+14,11.425263,360.694065,13.638690,0.310013,0.104566
1,UCI,METRO_TRAFFIC,Bottom_Up,Linear,BottomUp Linear,70.300547,0.198808,3.313466e-03,1026.251305,17.104188,22.485124,0.374752,1048.935237,17.482254,1.400426e+14,11.425263,360.694065,13.638690,0.310013,0.104566


# [UCI] PRSA_BEIJING

In [8]:
uci_prsa_beijing = get_df_for_dataset('UCI', 'PRSA_BEIJING')
uci_prsa_beijing_name = 'UCI PRSA_BEIJING'
assert not uci_prsa_beijing['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_prsa_beijing) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_prsa_beijing)}"

datasets.append(uci_prsa_beijing)

uci_prsa_beijing.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=0/UCI/PRSA_BEIJING


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
8,UCI,PRSA_BEIJING,Bottom_Up,Rank,BottomUp Rank,8.913546,0.185491,3.091514e-03,1670.633725,27.843895,20.362002,0.339367,1691.181218,28.186354,2.076114e+14,4.257235,124.059106,6.733149,0.706947,0.024013
0,UCI,PRSA_BEIJING,Bottom_Up,Cosine,BottomUp Cosine,25.000713,22.933039,3.822173e-01,1674.495577,27.908260,18.550826,0.309180,1715.979442,28.599657,2.033024e+14,4.318349,123.079215,6.853739,0.676113,0.024358
2,UCI,PRSA_BEIJING,Bottom_Up,L2,BottomUp L2,25.000713,0.124224,2.070399e-03,1671.366639,27.856111,18.446802,0.307447,1689.937665,28.165628,2.033024e+14,4.318349,123.079215,6.853739,0.676113,0.024358
10,UCI,PRSA_BEIJING,Window,Cosine,Window Cosine,25.000713,20.867567,3.477928e-01,1782.931196,29.715520,20.387477,0.339791,1824.186240,30.403104,2.033024e+14,4.318349,123.079215,6.853739,0.676113,0.024358
12,UCI,PRSA_BEIJING,Window,L2,Window L2,25.000713,0.189911,3.165185e-03,1757.808183,29.296803,19.983744,0.333062,1777.981838,29.633031,2.033024e+14,4.318349,123.079215,6.853739,0.676113,0.024358
17,UCI,PRSA_BEIJING,Window,L1,Window L1,25.000713,0.316836,5.280598e-03,1847.010137,30.783502,20.582662,0.343044,1867.909635,31.131827,2.033024e+14,4.318349,123.079215,6.853739,0.676113,0.024358
3,UCI,PRSA_BEIJING,Bottom_Up,RBF,BottomUp RBF,8.186200,114.889127,1.914819e+00,1406.167034,23.436117,20.064478,0.334408,1541.120639,25.685344,3.297307e+14,4.304336,132.630423,6.956408,0.684736,0.024278
7,UCI,PRSA_BEIJING,Bottom_Up,L1,BottomUp L1,8.186200,0.190278,3.171301e-03,1484.507729,24.741795,20.599555,0.343326,1505.297562,25.088293,3.297307e+14,4.304336,132.630423,6.956408,0.684736,0.024278
13,UCI,PRSA_BEIJING,Window,RBF,Window RBF,21.920192,111.867943,1.864466e+00,1742.936575,29.048943,19.102673,0.318378,1873.907191,31.231787,3.401965e+14,4.389293,123.869058,6.974675,0.676998,0.024758
22,UCI,PRSA_BEIJING,Bin_Seg,L2,BinSeg L2,35.368949,1.953722,3.256204e-02,1304.660800,21.744347,11.407577,0.190126,1318.022099,21.967035,4.233201e+14,4.352975,125.059803,6.998116,0.628950,0.024553


# [UCI] AIR_QUALITY

In [9]:
uci_air_quality = get_df_for_dataset('UCI', 'AIR_QUALITY')
uci_air_quality_name = 'UCI AIR_QUALITY'
assert not uci_air_quality['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_air_quality) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_air_quality)}"

datasets.append(uci_air_quality)

uci_air_quality.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=0/UCI/AIR_QUALITY
Error in outputs/report/seed=0/UCI/AIR_QUALITY/Bin_Seg/Clinear/2025-03-06_22-27-01/report.json: 'tuner_duration'


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
10,UCI,AIR_QUALITY,Window,Cosine,Window Cosine,10.754843,0.216216,3.603601e-03,299.878150,4.997969,5.335268,0.088921,305.429634,5.090494,5.300123e+12,28.098124,4743.731599,38.980345,0.186397,0.329089
28,UCI,AIR_QUALITY,Bin_Seg,Rank,BinSeg Rank,55.511022,0.175892,2.931531e-03,210.953061,3.515884,4.376185,0.072936,215.505138,3.591752,3.672935e+12,30.031312,5089.727093,39.310512,0.395658,0.351731
29,UCI,AIR_QUALITY,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000006,1.033147e-07,374.188192,6.236470,7.234854,0.120581,381.423052,6.357051,5.318583e+12,28.075026,5088.241817,39.749366,0.202422,0.328819
3,UCI,AIR_QUALITY,Bottom_Up,RBF,BottomUp RBF,55.377422,0.626822,1.044703e-02,228.895465,3.814924,4.248664,0.070811,233.770951,3.896183,3.827487e+12,30.404176,5350.012203,39.980412,0.393903,0.356098
20,UCI,AIR_QUALITY,Bin_Seg,Cosine,BinSeg Cosine,55.377422,28.558709,4.759785e-01,225.371310,3.756188,4.122224,0.068704,258.052243,4.300871,3.827487e+12,30.404176,5350.012203,39.980412,0.393903,0.356098
7,UCI,AIR_QUALITY,Bottom_Up,L1,BottomUp L1,55.377422,0.043207,7.201155e-04,228.825369,3.813756,4.120806,0.068680,232.989382,3.883156,3.827487e+12,30.404176,5350.012203,39.980412,0.393903,0.356098
21,UCI,AIR_QUALITY,Bin_Seg,Linear,BinSeg Linear,52.371409,0.487221,8.120350e-03,235.854777,3.930913,4.642321,0.077372,240.984319,4.016405,4.103682e+12,31.037103,5327.999942,40.043098,0.375991,0.363511
4,UCI,AIR_QUALITY,Bottom_Up,AR,BottomUp AR,55.778223,0.036471,6.078442e-04,196.906749,3.281779,4.656180,0.077603,201.599400,3.359990,3.609223e+12,32.123340,5685.888343,40.919999,0.400848,0.376233
1,UCI,AIR_QUALITY,Bottom_Up,Linear,BottomUp Linear,51.102204,0.030229,5.038142e-04,251.520920,4.192015,4.675243,0.077921,256.226392,4.270440,5.022154e+12,32.500930,5773.444482,41.326926,0.229887,0.380656
25,UCI,AIR_QUALITY,Bin_Seg,Normal,BinSeg Normal,55.911824,0.249189,4.153148e-03,244.790224,4.079837,4.326817,0.072114,249.366230,4.156104,4.045339e+12,33.307975,6068.354748,41.881480,0.346383,0.390108


# [INMET] SAOPAULO_SP

In [10]:
# inmet_saopaulo_sp = get_df_for_dataset('INMET', 'SAOPAULO_SP')
# inmet_saopaulo_sp_name = 'INMET SAOPAULO_SP'
# assert not inmet_saopaulo_sp['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
# assert len(inmet_saopaulo_sp) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(inmet_saopaulo_sp)}"

# datasets.append(inmet_saopaulo_sp)

# inmet_saopaulo_sp.sort_values(by='Avg_RMSE')

# Rank

In [11]:
def compute_arr(df, acc_d, algorithm_p, algorithm_q, dataset, metric):
    df_d = df[df['dataset_name'] == dataset]

    d_ap = df_d[df_d["cut_point_approach"] == algorithm_p]
    d_aq = df_d[df_d["cut_point_approach"] == algorithm_q]

    assert len(d_ap) == 1, f"Expected 1 line for approach {algorithm_p} but got {len(d_ap)}"
    assert len(d_aq) == 1, f"Expected 1 line for approach {algorithm_q} but got {len(d_aq)}"

    sr_ap = 1 / d_ap[metric].iloc[0]
    sr_aq = 1 / d_aq[metric].iloc[0]

    t_ap = d_ap["total_duration"].iloc[0]
    t_aq = d_aq["total_duration"].iloc[0]

    sr_ratio = sr_ap / sr_aq if sr_aq != 0 else np.nan  # Avoid division by zero
    time_ratio = np.log(t_ap / t_aq) if t_aq != 0 else np.nan  # Avoid division by zero

    arr = sr_ratio / (1 + acc_d * time_ratio)
    
    return arr


def compute_arr_ranking(df, acc_d, metric):
    algorithms = df["cut_point_approach"].unique()
    datasets = df['dataset_name'].unique()
    arr_df = pd.DataFrame({
        'dataset': [],
        'ap': [],
        'aq': [],
        'arr': [],
    })

    for dataset in datasets:
        for ap in algorithms:
            for aq in algorithms:
                if ap == aq:
                    continue

                arr_df = pd.concat([arr_df, pd.DataFrame({
                    'dataset': [dataset],
                    'ap': [ap],
                    'aq': [aq],
                    'arr': [compute_arr(df, acc_d, ap, aq, dataset, metric)],
                })])

    grouped = arr_df.groupby(['ap', 'aq'])['arr'].apply(lambda x: np.prod(x) ** (1 / len(x))).reset_index()

    ranking_df = grouped.groupby('ap')['arr'].mean().reset_index()
    ranking_df['rank'] = ranking_df['arr'].rank(ascending=False, method='max')

    return ranking_df.sort_values(by='rank')

In [12]:
metrics_df = pd.concat(datasets, axis=0)
metrics_df = metrics_df[~metrics_df['cut_point_approach'].isin(DROP_APPROACHES)]
metrics_df['dataset_name'] = metrics_df['database'] + ' ' + metrics_df['dataset']

## WAPE

In [13]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.151844,1.0
2,BinSeg L1,1.135139,2.0
18,BottomUp Rank,1.127959,3.0
22,Window Cosine,1.127257,4.0
19,Full,1.111458,5.0
21,Window Clinear,1.100142,6.0
17,BottomUp RBF,1.100139,7.0
28,Window RBF,1.083953,8.0
12,BottomUp L1,1.073209,9.0
3,BinSeg L2,1.064375,10.0


In [14]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.148878,1.0
2,BinSeg L1,1.130729,2.0
18,BottomUp Rank,1.114093,3.0
22,Window Cosine,1.109108,4.0
21,Window Clinear,1.094135,5.0
17,BottomUp RBF,1.091760,6.0
20,Window AR,1.089534,7.0
19,Full,1.086436,8.0
16,BottomUp Normal,1.079354,9.0
28,Window RBF,1.077816,10.0


In [15]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.323655,1.0
16,BottomUp Normal,1.266442,2.0
20,Window AR,1.241551,3.0
6,BinSeg Normal,1.183824,4.0
4,BinSeg Linear,1.170963,5.0
13,BottomUp L2,1.162281,6.0
14,BottomUp Linear,1.155297,7.0
2,BinSeg L1,1.133043,8.0
23,Window L1,1.107179,9.0
21,Window Clinear,1.091968,10.0


In [16]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.419565,1.0
16,BottomUp Normal,2.097713,2.0
20,Window AR,1.785621,3.0
6,BinSeg Normal,1.734541,4.0
4,BinSeg Linear,1.725959,5.0
10,BottomUp Clinear,1.597601,6.0
25,Window Linear,1.376190,7.0
26,Window Mahalanobis,1.274360,8.0
29,Window Rank,1.197608,9.0
13,BottomUp L2,1.183846,10.0


In [17]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
20,Window AR,2.156208,1.0
14,BottomUp Linear,2.135847,2.0
25,Window Linear,1.868537,3.0
26,Window Mahalanobis,1.621691,4.0
27,Window Normal,1.467194,5.0
4,BinSeg Linear,1.321008,6.0
29,Window Rank,1.319626,7.0
16,BottomUp Normal,1.305691,8.0
2,BinSeg L1,1.228337,9.0
6,BinSeg Normal,1.145923,10.0


## MSE

In [18]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.293039,1.0
18,BottomUp Rank,1.244830,2.0
22,Window Cosine,1.231668,3.0
2,BinSeg L1,1.216934,4.0
19,Full,1.180968,5.0
17,BottomUp RBF,1.179830,6.0
8,BinSeg Rank,1.130716,7.0
21,Window Clinear,1.124368,8.0
1,BinSeg Cosine,1.124047,9.0
3,BinSeg L2,1.121023,10.0


In [19]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.288754,1.0
18,BottomUp Rank,1.228673,2.0
2,BinSeg L1,1.211315,3.0
22,Window Cosine,1.211014,4.0
17,BottomUp RBF,1.170001,5.0
16,BottomUp Normal,1.159026,6.0
19,Full,1.153598,7.0
20,Window AR,1.133179,8.0
8,BinSeg Rank,1.126733,9.0
21,Window Clinear,1.117394,10.0


In [20]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,1.353845,1.0
13,BottomUp L2,1.300304,2.0
20,Window AR,1.286574,3.0
6,BinSeg Normal,1.247509,4.0
10,BottomUp Clinear,1.243420,5.0
2,BinSeg L1,1.211004,6.0
18,BottomUp Rank,1.197320,7.0
4,BinSeg Linear,1.184707,8.0
22,Window Cosine,1.165466,9.0
17,BottomUp RBF,1.155636,10.0


In [21]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,2.236735,1.0
14,BottomUp Linear,2.194647,2.0
20,Window AR,1.848811,3.0
6,BinSeg Normal,1.836029,4.0
4,BinSeg Linear,1.743515,5.0
10,BottomUp Clinear,1.480209,6.0
13,BottomUp L2,1.325381,7.0
25,Window Linear,1.243697,8.0
2,BinSeg L1,1.211878,9.0
27,Window Normal,1.154347,10.0


In [22]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
20,Window AR,2.221416,1.0
14,BottomUp Linear,1.886116,2.0
25,Window Linear,1.673126,3.0
27,Window Normal,1.493585,4.0
16,BottomUp Normal,1.448861,5.0
4,BinSeg Linear,1.386990,6.0
26,Window Mahalanobis,1.336574,7.0
2,BinSeg L1,1.310458,8.0
6,BinSeg Normal,1.258225,9.0
13,BottomUp L2,1.253146,10.0


## RMSE

In [23]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.143841,1.0
18,BottomUp Rank,1.125417,2.0
2,BinSeg L1,1.113362,3.0
22,Window Cosine,1.089799,4.0
21,Window Clinear,1.084433,5.0
17,BottomUp RBF,1.078324,6.0
20,Window AR,1.062107,7.0
19,Full,1.058764,8.0
3,BinSeg L2,1.055662,9.0
12,BottomUp L1,1.053085,10.0


In [24]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
13,BottomUp L2,1.141170,1.0
18,BottomUp Rank,1.111842,2.0
2,BinSeg L1,1.109294,3.0
20,Window AR,1.096656,4.0
16,BottomUp Normal,1.087748,5.0
21,Window Clinear,1.078767,6.0
22,Window Cosine,1.072481,7.0
17,BottomUp RBF,1.070360,8.0
12,BottomUp L1,1.045934,9.0
3,BinSeg L2,1.043753,10.0


In [25]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.360589,1.0
16,BottomUp Normal,1.277778,2.0
20,Window AR,1.250968,3.0
6,BinSeg Normal,1.215504,4.0
4,BinSeg Linear,1.198133,5.0
14,BottomUp Linear,1.171784,6.0
13,BottomUp L2,1.155404,7.0
2,BinSeg L1,1.112320,8.0
18,BottomUp Rank,1.086560,9.0
23,Window L1,1.081092,10.0


In [26]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.453237,1.0
16,BottomUp Normal,2.123715,2.0
20,Window AR,1.799034,3.0
6,BinSeg Normal,1.779656,4.0
4,BinSeg Linear,1.764714,5.0
10,BottomUp Clinear,1.651643,6.0
25,Window Linear,1.393697,7.0
26,Window Mahalanobis,1.251706,8.0
29,Window Rank,1.213327,9.0
13,BottomUp L2,1.175954,10.0


In [27]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
20,Window AR,2.183150,1.0
14,BottomUp Linear,2.180302,2.0
25,Window Linear,1.882164,3.0
26,Window Mahalanobis,1.600463,4.0
27,Window Normal,1.530322,5.0
29,Window Rank,1.348085,6.0
4,BinSeg Linear,1.320344,7.0
16,BottomUp Normal,1.286316,8.0
2,BinSeg L1,1.205100,9.0
6,BinSeg Normal,1.149366,10.0
